In [114]:
# !pip install -U langchain-openai
# !pip install python-dotenv
# ! pip install -U langgraph
! pip install log_feedback

ERROR: Could not find a version that satisfies the requirement log_feedback (from versions: none)
ERROR: No matching distribution found for log_feedback


In [77]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

## Step 1: Ingest PDFs into ChromaDB

In [78]:
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.environ["OPENAI_API_KEY"]

# Setup paths
pdf_folder = "pdf_docs"
chroma_db_dir = "chroma_db"

# Load and process all PDFs
all_docs = []
for file in os.listdir(pdf_folder):
    if file.endswith(".pdf"):
        path = os.path.join(pdf_folder, file)
        print(f"Loading: {file}")
        loader = PyPDFLoader(path)
        docs = loader.load()
        for doc in docs:
            doc.metadata["source"] = file
        all_docs.extend(docs)

print(f"Loaded {len(all_docs)} documents.")

# Split into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = splitter.split_documents(all_docs)
print(f"Split into {len(chunks)} chunks.")

# Embed and store in ChromaDB
embedding_model = OpenAIEmbeddings(api_key=openai_api_key)
vectordb = Chroma.from_documents(chunks, embedding=embedding_model, persist_directory=chroma_db_dir)
vectordb.persist()
print("ChromaDB vector store created and persisted.")


Loading: H20 GAS PLASMA STERILIZERS USER MANUAL.pdf
Loading: Hidrojen Peroksit DIS ERYIGIT KISA SUNUM.pdf
Loading: HIDROJEN PEROKSIT DUSUK ISI STERILIZATORU.pdf
Loading: HIDROJEN PEROKSIT RAKIPLER KIYASLAMA.pdf
Loading: HP SISTEMI SUNUM ERYIGIT.pdf
Loading: Hydrogen Peroxide Service Video.pdf
Loading: TD.03.02.05 GP H2O2 Sterilizer and Cartridge User Manual.pdf
Loaded 143 documents.
Split into 492 chunks.
ChromaDB vector store created and persisted.


In [102]:
# LLM'yi başlat
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, api_key=openai_api_key)

In [103]:
# Chroma vektör veritabanı
vectordb = Chroma(persist_directory="chroma_db", embedding_function=OpenAIEmbeddings(api_key=openai_api_key))
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

## Step 2: Define the LangGraph Router and Agent Nodes

Set up a LangGraph router that detects intent: "instruction", "safety", or "escalation"  
    Define agent nodes that act accordingly  
    Use the ChromaDB vector store we just created

## Adım 2A: Soru Yönlendirici Fonksiyonu (Router Node)

Bu fonksiyon, gelen kullanıcı sorusunun hangi türde olduğunu belirler:  
"instruction" → Talimat/İşlem (örn. “Nasıl çalıştırılır?”)  
"safety" → Güvenlik/Teknik tehlike (örn. “Sızıntı olursa ne yapmalıyım?”)  
"escalation" → Belirsiz ya da uzman müdahalesi gereken durumlar

In [104]:
def route_query(state):
    query = state.input  # ✅ dict değil, Pydantic model
    # Anahtar kelimeler
    talimat_kelimeleri = ["nasıl", "başlat", "yükle", "döngü", "çalıştır", "ekle", "boşalt"]
    guvenlik_kelimeleri = ["sızıntı", "alarm", "koku", "tehlike", "dökülme", "acil", "patlama"]

    query_lower = query.lower()
    if any(kelime in query_lower for kelime in guvenlik_kelimeleri):
        state.agent_type = "safety"
    elif any(kelime in query_lower for kelime in talimat_kelimeleri):
        state.agent_type = "instruction"
    else:
        state.agent_type = "escalation"

    return state  # ✅ Geriye state nesnesini döndür!


## Adım 2B: Ajan Düğümlerinin Tanımı

Her ajan, aşağıdakileri yapar:  
ChromaDB’den ilgili belgeleri getirir (retriever)  
OpenAI üzerinden yanıt oluşturur  
Türkçe ve göreve özel bir biçimde cevap döner

In [105]:
def safety_agent(state):
    query = state.input

    prompt = PromptTemplate.from_template(
    """Aşağıdaki içerikler sterilizatör kılavuzundan alınmıştır:

    {context}

    Soru: {question}

    Lütfen yalnızca yukarıdaki içeriğe dayanarak Türkçe ve açık bir şekilde cevap ver."""
    )

    chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=False  # İsterseniz True yapabilirsiniz
    )

    result = chain.invoke(query)
    state.output = result
    return state


In [106]:
def instruction_agent(state):
    query = state.input

    prompt = PromptTemplate.from_template(
        """Aşağıdaki içerikler sterilizatör kılavuzundan alınmıştır:

{context}

Soru: {question}

Lütfen yalnızca yukarıdaki içeriğe dayanarak Türkçe ve açık bir şekilde cevap ver."""
    )

    chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        chain_type_kwargs={"prompt": prompt},
        return_source_documents=False
    )

    result = chain.invoke(query)
    state.output = result
    return state



In [107]:
def escalation_agent(state):
    state.output = (
        "Bu soruya mevcut belgelerle kesin bir cevap veremiyorum. "
        "Lütfen yetkili bir teknik uzmana danışınız veya kullanıcı kılavuzunu gözden geçiriniz."
    )
    return state


## Adım 2C: LangGraph Akış Grafiği Kurulumu

In [108]:
from langgraph.graph import StateGraph, END
from pydantic import BaseModel

from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [109]:
# Pydantic veri modeli
class ChatState(BaseModel):
    input: str
    agent_type: str = None
    output: str = None

In [110]:
# Graf Düğümü Tanımlama ve Akışı Kurma
# Grafik nesnesini oluştur
workflow = StateGraph(ChatState)

# Düğümleri ekle
workflow.add_node("router", route_query)
workflow.add_node("instruction", instruction_agent)
workflow.add_node("safety", safety_agent)
workflow.add_node("escalation", escalation_agent)

# Akışı tanımla
workflow.set_entry_point("router")
workflow.add_conditional_edges(
    "router",
    lambda state: state.agent_type,  # ✅ Pydantic modeli olduğu için .agent_type kullanıyoruz
    {
        "instruction": "instruction",
        "safety": "safety",
        "escalation": "escalation"
    }
)



# Ajan düğümlerinden sonra sona git
workflow.add_edge("instruction", END)
workflow.add_edge("safety", END)
workflow.add_edge("escalation", END)

# Grafı derle
app = workflow.compile()


In [112]:
result = app.invoke({"input": "Cihazı nasıl başlatırım?"})
print(" Cevap:", result["output"])

👉 Cevap: {'query': 'Cihazı nasıl başlatırım?', 'result': 'Cihazı başlatmak için cihazın açma anahtarını sağ yöne çevirerek açabilirsiniz. Cihaz açıldıktan sonra ekran görünecek ve MENÜ ve AYARLAR butonlarına erişebileceksiniz. Menü butonuna basarak cihazla ilgili yapılabilecek kontrolleri görebilir ve cihazı başlatabilirsiniz.'}


## 3. Loglama Fonksiyonu

In [115]:
from log_feedback import initialize_log, log_interaction

initialize_log()

soru = "Cihazı nasıl başlatırım?"
sonuc = app.invoke({"input": soru})
cevap = sonuc["output"]
ajan = sonuc["agent_type"]

# Simüle edilmiş kullanıcı geri bildirimi
geri_bildirim = "👍"
yorum = "Gayet netti."

log_interaction(soru, ajan, cevap, geri_bildirim, yorum)

